### Lime Anchor - Experiment for Text

In [ ]:
import anchor

import pandas as pd 
import sklearn

import string
import re


import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from pprint import pprint
from time import time
import logging

from sklearn.datasets import fetch_20newsgroups

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

In [ ]:

newsgroups = fetch_20newsgroups(subset='all', shuffle=False, remove=('headers', 'footers', 'quotes'))

df = pd.DataFrame([newsgroups.data, newsgroups.target.tolist()]).T
df.columns = ['text', 'target']

targets = pd.DataFrame( newsgroups.target_names)
targets.columns=['title']

newsgroups_full = pd.merge(df, targets, left_on='target', right_index=True)
newsgroups_full['date'] = pd.to_datetime('now')
newsgroups_full = newsgroups_full.reset_index(drop=True)

import gc
df = []
gc.collect()

In [ ]:
newsgroups_full.head()

In [ ]:


def cleanse_punctuation_whitespace_stopwords(df):
    import string 
        
    #text already appears to be cleansed ... but, just making sure. limited pre-processing - these assumptions can be played around with when we build classifiers
    df['text'] = df['text'].str.lower()
    df['text'] = df['text'].str.replace('\d+', '')

    def remove_punctuations(text):

        for punctuation in string.punctuation:

            text = text.replace(punctuation, ' ')

        return text


    df['text'] = df['text'].apply(remove_punctuations)  #remove punctuation


    #Adding this due to finding below that pre-cleansed text is corrupt. (contains cases like can t) . Remove single stand-alone characters. ("a", "e", etc)

    df['text'] = df['text'].str.replace(r'\b(?<=)[a-z](?=)\b', ' ') #remove single stand-alone characters.




    #research_article_corpus['covid_articles'] = metadata_english[metadata_english["wc"] >= 10]

    def strip_extra_whitespace(doc):
            ws_stripped = " ".join(doc.split())
            return ws_stripped


    df['text']  = df['text'] .apply(strip_extra_whitespace)

    import re
    
    

    def rm_s(doc):
        from nltk.corpus import stopwords
        import re
        
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
        text = pattern.sub('', doc)
        return text

    from multiprocessing import Pool

   


    df['text'] = df['text'].apply(rm_s)
    #p.close()
    #p.terminate()
    return df 

newsgroups_full = cleanse_punctuation_whitespace_stopwords(newsgroups_full)

In [ ]:
newsgroups_full = newsgroups_full.reset_index(drop=True)

In [ ]:


#research_article_corpus['covid_articles'] = metadata_english[metadata_english["wc"] >= 10]


import re
from nltk.corpus import stopwords
pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')

def rm_s(doc):
    text = pattern.sub('', doc)
    return text

from multiprocessing import Pool

p = Pool(11)


newsgroups_full['text'] = p.map(rm_s, newsgroups_full['text'])
p.close()
p.terminate()

In [ ]:


from nltk.stem.snowball import PorterStemmer
from multiprocessing import Pool
from nltk.stem.porter import *
from nltk.stem.porter import *
import gc

p = Pool(11)
        
def stemmer_fn(text): 
    from nltk.stem import SnowballStemmer
    
    stemmer = SnowballStemmer("english")
    stemmed = [stemmer.stem(word) for word in text.split()]
 
 
    return stemmed

newsgroups_full['text']= p.map(stemmer_fn, newsgroups_full['text'])


p.close()
p.terminate

from nltk.tokenize.treebank import TreebankWordDetokenizer

def detokenizer(doc):
    doc_detokenized = TreebankWordDetokenizer().detokenize(doc)
    return doc_detokenized
p = Pool(11)

newsgroups_full["text"] =  p.map(detokenizer,newsgroups_full["text"])



In [ ]:
df = []
gc.collect()

In [ ]:

def wordcount_fn(df):
    df["wc"] = df['text'].str.count(' ') + 1
    df["wc"].hist(bins=1000)

    print('max word count:',  df["wc"].max())
    print('mean word count:',  df["wc"].mean())
    print('min word count:',  df["wc"].min())

    print('-------')
    print('total # of (non-null) documents:',  len(df[df['text'].isnull() == False]))
    print('total # of (non-null) documents less than 10 words:',  len(df[df['wc'] < 10]))
    return df
    
newsgroups_full = wordcount_fn(newsgroups_full)

In [ ]:
newsgroups_full = newsgroups_full[newsgroups_full['wc'] > 10]
newsgroups_full.shape

In [ ]:
newsgroups_full.to_csv("newsgroups_full_wc10.csv", index=False)

In [2]:
import pandas as pd 
newsgroups_full = pd.read_csv("newsgroups_full_wc10.csv")
newsgroups_full.head()

,text,target,title,date,wc
0,morgan guzman era run higher last year cub idi...,9,rec.sport.baseball,2020-04-06 21:11:38.543427,18
1,um header said career hodapp manag pa nine yea...,9,rec.sport.baseball,2020-04-06 21:11:38.543427,72
2,dalla alot nerv say wfan bunch hoodlum met fan...,9,rec.sport.baseball,2020-04-06 21:11:38.543427,50
3,interest uncov statist boston red sox player m...,9,rec.sport.baseball,2020-04-06 21:11:38.543427,37
4,face philli team got start honest think citi h...,9,rec.sport.baseball,2020-04-06 21:11:38.543427,87


In [3]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(newsgroups_full['text'], newsgroups_full['target'],  shuffle=True, random_state=42)




In [4]:
X_train.shape

(12561,)

In [5]:
X_test.shape

(4188,)

In [9]:
import gc
newsgroups_full = []
df = []
gc.collect()

360

In [10]:
grid =[]
import gc
gc.collect()

20

In [11]:

# #############################################################################
# Define a pipeline combining a text feature extractor with a simple
# classifier
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(n_jobs=-1,  random_state=3)),
])
# increase processing time in a combinatorial way
parameters = {
    'vect__min_df': [2, 5, 10],
    'vect__max_df': [0.9, 1],
    # 'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': [(1, 1), (1, 2), (1,3)],  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': (None , 'l1', 'l2'),
    'clf__bootstrap': [True, False],
    'clf__class_weight': ['balanced', 'balanced_subsample'],
    'clf__max_depth': [10, 100, 500, None],
    'clf__max_features': ['auto', 'sqrt'],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__min_samples_split': [2, 5, 10],
    'clf__n_estimators': [200, 400, 600, 800, 1000, 1200, 1600, 1800, 2000]}

grid = RandomizedSearchCV(pipeline, parameters, n_jobs=11, scoring = 'f1_weighted', cv=3, random_state=0, n_iter=150, verbose=20)

grid.fit( X_train, pd.Categorical(y_train).codes)

Fitting 3 folds for each of 150 candidates, totalling 450 fits


[Parallel(n_jobs=11)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=11)]: Done   1 tasks      | elapsed:    7.3s
[Parallel(n_jobs=11)]: Done   2 tasks      | elapsed:    7.6s
[Parallel(n_jobs=11)]: Done   3 tasks      | elapsed:    8.5s
[Parallel(n_jobs=11)]: Done   4 tasks      | elapsed:   13.5s
[Parallel(n_jobs=11)]: Done   5 tasks      | elapsed:   13.8s
[Parallel(n_jobs=11)]: Done   6 tasks      | elapsed:   14.3s
[Parallel(n_jobs=11)]: Done   7 tasks      | elapsed:   14.3s
[Parallel(n_jobs=11)]: Done   8 tasks      | elapsed:   14.4s
[Parallel(n_jobs=11)]: Done   9 tasks      | elapsed:   14.6s
[Parallel(n_jobs=11)]: Done  10 tasks      | elapsed:   20.0s
[Parallel(n_jobs=11)]: Done  11 tasks      | elapsed:   20.2s
[Parallel(n_jobs=11)]: Done  12 tasks      | elapsed:   20.5s
[Parallel(n_jobs=11)]: Done  13 tasks      | elapsed:   21.2s
[Parallel(n_jobs=11)]: Done  14 tasks      | elapsed:   21.5s
[Parallel(n_jobs=11)]: Done  15 tasks      | elapsed:  

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              CountVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.int64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                          

In [12]:
grid.best_params_

{'vect__ngram_range': (1, 2),
 'vect__min_df': 2,
 'vect__max_df': 0.9,
 'tfidf__use_idf': False,
 'tfidf__norm': 'l1',
 'clf__n_estimators': 2000,
 'clf__min_samples_split': 5,
 'clf__min_samples_leaf': 2,
 'clf__max_features': 'auto',
 'clf__max_depth': None,
 'clf__class_weight': 'balanced_subsample',
 'clf__bootstrap': False}

### Develop Baseline
    - SVM

In [21]:

>>> from sklearn.linear_model import SGDClassifier
>>> text_clf = Pipeline([
...     ('vect', CountVectorizer(ngram_range= (1, 1), min_df = 2, max_df=1.0 )),
...     ('tfidf', TfidfTransformer(use_idf=False, norm='l1')),
...     ('clf', SGDClassifier(loss='hinge', penalty='l2',
...                           alpha=1e-3, random_state=42,
...                           max_iter=5, tol=None, class_weight='balanced')),
... ])


text_clf.fit( X_train, pd.Categorical(y_train).codes)
from sklearn.metrics import classification_report
y_pred = text_clf.predict(X_test)
print(classification_report(pd.Categorical(y_test).codes, pd.Categorical(y_pred).codes))

              precision    recall  f1-score   support

           0       0.56      0.50      0.53       161
           1       0.65      0.71      0.68       222
           2       0.68      0.63      0.66       218
           3       0.73      0.53      0.61       205
           4       0.61      0.77      0.68       213
           5       0.88      0.77      0.82       248
           6       0.77      0.80      0.78       233
           7       0.62      0.87      0.72       212
           8       0.76      0.84      0.80       217
           9       0.88      0.83      0.85       214
          10       0.80      0.95      0.87       222
          11       0.78      0.77      0.77       220
          12       0.71      0.52      0.60       221
          13       0.85      0.85      0.85       218
          14       0.76      0.78      0.77       224
          15       0.71      0.78      0.75       227
          16       0.63      0.79      0.70       203
          17       0.80    

In [13]:
from sklearn.metrics import classification_report

pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range= (1, 2), min_df = 2, max_df=0.9 )),
    ('tfidf', TfidfTransformer(use_idf=False, norm='l1')),
    ('clf', RandomForestClassifier(n_estimators= 2000, min_samples_split=5, min_samples_leaf=2, max_features='auto', max_depth=None, class_weight='balanced_subsample', bootstrap=False, n_jobs=-1,  random_state=3))])

pipeline.fit( X_train, pd.Categorical(y_train).codes)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.9,
                                 max_features=None, min_df=2,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 RandomForestClassifier(bootstrap=False, ccp_alpha=0.0,
                                        class_weight='balanced_subsample',
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes

In [15]:
from sklearn.metrics import classification_report
y_pred = pipeline.predict(X_test)
print(classification_report(pd.Categorical(y_test).codes, pd.Categorical(y_pred).codes))

              precision    recall  f1-score   support

           0       0.54      0.47      0.50       161
           1       0.67      0.69      0.68       222
           2       0.70      0.73      0.72       218
           3       0.66      0.69      0.67       205
           4       0.75      0.78      0.77       213
           5       0.88      0.83      0.86       248
           6       0.79      0.78      0.78       233
           7       0.76      0.78      0.77       212
           8       0.76      0.82      0.79       217
           9       0.83      0.86      0.84       214
          10       0.86      0.92      0.89       222
          11       0.92      0.76      0.83       220
          12       0.64      0.62      0.63       221
          13       0.83      0.87      0.85       218
          14       0.83      0.78      0.80       224
          15       0.64      0.82      0.72       227
          16       0.67      0.78      0.72       203
          17       0.90    

In [24]:



import anchor

import sys
import spacy

nlp = spacy.load("en_core_web_lg")


In [33]:
from anchor import utils
from anchor import anchor_text

In [69]:
from anchor import anchor_text
explainer = anchor_text.AnchorText(nlp, np.arange(20), use_unk_distribution=False)

def predict_pipeline(texts):
    return pipeline.predict(texts)



In [96]:
X_test[5456]

'understand new gps box option known differenti readi appar land base beacon tranmit gps correct inform gps receiv differenti option instal system work frequenc use land base beacon thank advanc'

In [98]:
X_test[15699]

'think assumpt turn softwar virus prove fact court get damag theoret supposs abl get damag give evid virus came softwar sinc comput compani defend uninvolv proven guilti pleas pascal nooooo'

In [99]:
np.random.seed(1)

text =  X_test[15699]
pred = explainer.class_names[predict_pipeline([text])[0]]
alternative =  explainer.class_names[1 - predict_pipeline([text])[0]]
print('Prediction: %s' % pred)


Prediction: 1


In [100]:
exp = explainer.explain_instance(text, predict_pipeline, threshold=0.95, use_proba=True)

/home/spenser/.conda/envs/gpu/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  "__main__", mod_spec)
/home/spenser/.conda/envs/gpu/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  "__main__", mod_spec)
/home/spenser/.conda/envs/gpu/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  "__main__", mod_spec)
/home/spenser/.conda/envs/gpu/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  "__main__", mod_spec)
/home/spenser/.conda/envs/gpu/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  "__main__", mod_spec)
/home/spenser/.conda/envs/gpu/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  "__main__", mod_spec)
/home/spenser/.conda/envs/gpu/lib/python3.7/runpy.py:193: UserWarning:

In [101]:

print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()


Anchor: virus AND fact
Precision: 0.96

Examples where anchor applies and model predicts 1:

put assumpt appear softwar alert have fact matter 're damag theoret supposs abl get damag agree evid virus worked softwar sinc comput compani agree uninvolv acknowledged guilti warrants pascal nooooo
excuse assumpt seem softwar epidemic appear fact ruling 're damag theoret supposs abl get damag have evid virus shocked softwar sinc comput compani agree uninvolv established guilti charges pascal nooooo
realize assumpt appear softwar variant suggest fact judgment 're damag theoret supposs abl get damag appreciate evid virus saw softwar sinc comput compani oppose uninvolv committed guilti sympathies pascal nooooo
look assumpt appear softwar fungal seem fact witness 're damag theoret supposs abl get damag have evid virus knew softwar sinc comput compani insist uninvolv been guilti suggestions pascal nooooo
guess assumpt have softwar vulnerability seem fact county have damag theoret supposs abl get d

In [108]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

Anchor: virus AND fact
Precision: 0.96
Coverage: 0.00


In [112]:
 exp.features()

[17, 6]

In [114]:
exp.names()

['virus', 'fact']

In [121]:
pd.DaataFrame.from_dict(pipeline.named_steps.vect.vocabulary_, columns = 'count'

{'netter': 114922,
 'got': 71270,
 'use': 184016,
 'acceler': 557,
 'fpu': 64597,
 'wonder': 193911,
 'point': 130772,
 'anyon': 7426,
 'know': 90903,
 'current': 39156,
 'driver': 48576,
 'version': 186873,
 'handl': 74582,
 'bit': 18310,
 'grayscal': 72407,
 'card': 24264,
 'get': 67748,
 'video': 187368,
 'option': 121888,
 'would': 195196,
 'hate': 75368,
 'hard': 74990,
 'drive': 48242,
 'time': 176196,
 'need': 114152,
 'new': 115283,
 'make': 101776,
 'thank': 173569,
 'jason': 87179,
 'got use': 71469,
 'point anyon': 130790,
 'anyon know': 7523,
 'know current': 91030,
 'current driver': 39200,
 'driver version': 48722,
 'handl bit': 74588,
 'card get': 24340,
 'get video': 68506,
 'video option': 187404,
 'option would': 121961,
 'would hate': 195501,
 'hate hard': 75382,
 'hard drive': 75020,
 'drive use': 48538,
 'drive time': 48523,
 'time need': 176600,
 'need new': 114398,
 'new driver': 115391,
 'driver drive': 48616,
 'drive make': 48394,
 'thank jason': 173640,
 'expe

In [ ]:
# Get test examples where the anchora pplies


fit_anchor = np.where(np.all(X_test[:, exp.names()] == X_test[idx][exp.names()], axis=1))[0]
print('Anchor test coverage: %.2f' % (fit_anchor.shape[0] / float(dataset.test.shape[0])))
print('Anchor test precision: %.2f' % (np.mean(predict_fn(dataset.test[fit_anchor]) == predict_fn(dataset.test[idx].reshape(1, -1)))))


print('Partial anchor: %s' % (' AND '.join(exp.names(1))))
print('Partial precision: %.2f' % exp.precision(1))
print('Partial coverage: %.2f' % exp.coverage(1))

In [ ]:

print('Partial anchor: %s' % (' AND '.join(exp.names(0))))
print('Precision: %.2f' % exp.precision(0))
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))